# Постановка задачи
Обучить модель yolo11 на полученных данных для задачи распознавания посуды, еды и людей в кадре. Делаем упрощение, что мы не отслеживаем никакую мебель, кроме стола за которым ведется ведется трапеза, так как отслеживание стола имеет конкретный прикладной характер. Так же не остлеживаем столовые приборы и сторонние объекты в кадре по типу телефона. 

# Логика решения 

1. Найти подходящую модель-трекер для авто разметки, уже есть готовые решения, смотрим, какое подойдет лучше всего в инфраструктуре yolo (Лучше всего подошло yolo11s)
2. Разметить данные через эту модель, посмотреть результаты, свести ручную разметку к возможному минимуму. 

# Скачиваем датасет

In [ ]:
import requests
from time import time

public_key = "https://disk.360.yandex.ru/d/_8CpYlNBpmYidQ"

print("Запрашиваем ссылку на скачивание...")
api_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download"
params = {"public_key": public_key}

response = requests.get(api_url, params=params)
response.raise_for_status()
download_url = response.json()["href"]
print("Ссылка получена, начинаем скачивание...")

output_path = "dataset.zip"
with requests.get(download_url, stream=True) as r:
    r.raise_for_status()
    total_size = int(r.headers.get('content-length', 0))
    downloaded = 0
    chunk_size = 8192
    start = time()

    with open(output_path, "wb") as f:
        for i, chunk in enumerate(r.iter_content(chunk_size=chunk_size), 1):
            if chunk:
                f.write(chunk)
                downloaded += len(chunk)
                if i % 100 == 0 or downloaded == total_size:
                    percent = downloaded * 100 / total_size if total_size else 0
                    speed = downloaded / (time() - start + 1e-3) / 1024  # KB/s
                    print(f"Скачано: {downloaded / 1024:.1f} KB ({percent:.1f}%), Скорость: {speed:.1f} KB/s")

print("✅ Файл успешно скачан:", output_path)


# Тестируем разные трекеры 

Сначала протестируем из документации 

1. botsort.yaml
2. bytetrack.yaml

Так же посмотрим разные типы моделей, какая из них лучше определяет предметы. 

Нужно определять 

1. Людей в кадре
2. Тарелки
3. Еду 

Людей в кадре и тарелки (а так же столовые приборы) хорошо определяют базовые трекеры, с едой проблемы, ищем подходящий. 



In [6]:
from ultralytics import YOLO

model = YOLO("yolo11x-seg.pt")  # Load an official Detect model

# Записываем результаты по размерам модели 

*сейчас мы выбираем модель для авто разметки, поэтому первая необходимость - задетектить как можно больше объектов в кадре. Желательно правильно.

yolo11n с bytetrack - детектит только крупные объекты
yolo11s с bytetrack - детектит все мелкие предметы, даже попробовал определить еду как пиццу.
yolo11m с bytetrack - не детектит некоторые большие объекты, пытается детектит мелкие, получается хуже чем у s
yolo11l с bytetrack - так же пропускает некоторые крупные объекты, детектит больше мелких объектов, но качестве хуже чем у s
yolo11x с bytetrack - как ни странно качество как у n, но чуть лучше 

С bytetrack лучшая модель __yolo11s__

Дальше тестим с botsort
yolo11n - видит больше крупных объектов
yolo11s - видит практически все, некоторые объекты пропадают, но в целом это уже можно использовать для автоматизации.
yolo11m - меньше объектов, хуже s
yolo11l - хуже s
yolo11x - хуже s

Лучшая модель с botsort __yolo11s__

Без разницы какой трекер, одинаковые предметы в кадре не видит. 

# Теперь пройдемся по видео этой моделью, чтобы определить, какие видео придется все таки размечать руками 

* 1.mov - вообще не видит кружек, человека задетектило, из смешного - одну кружку в процессе задетектило как wife. Придется размечать самостоятельно все.
* 2_1.mov - в начале не увидел кружки по середине, зато даже подсветил броколи в тарелке, что явно плюс, увидел человека, после чего начал видеть на одну кружку в центре больше и потерял * * броколи, но не увидел поднос, зато увидел соус. На этом видео модель хорошо поможет и ускорит процесс
* 3_1.mov - видит людей в кадре, не видит только кружку по середине и подносы, в некоторых кадрах видит телефон и пытается поймать ложки. Не видит вилку на тарелке, одну тарелку считает фруктом. В целом по этому видео так же хорошо поможет и ускорит процесс разметки. 
* 3_2.mov - аналогичная ситуация с предыдущем видео, не видит две тарелки у людей, вместо одной тарелки детектит пиццу, столовые приборы в тарелке тоже не видит, но если они лежат отдельно, то он их детектит. Так же поможет частично.
* 4_1.mov - не видит поднос и прибор на нем, перестает видеть кружки в центре, с этим у него проблема, человека видит и то что у него в руках тоже, что интересно, поднос он начинает видеть когда он у человека в руках. Кружки под конец перестал видеть вообще. Начало разметки автоматизирует, остальное придется руками делать
* 4.mov - не видит кружки в центре и поднос, человека видит, не видит что у него в руках. Аналогично с предыдущим поможет автоматизировать начало разметки. 

Для теста будем использовать видео 4_1, т.к. на нем много обектов, плюс они начинают исчезать, что хорошо покажет способность к обобщению.

# Общий вывод 

Модель поможет автоматизировать начало разметки, после чего можно будет добить руками, что сильно сократит время

Имена файлов переименуем 


4 > test, на нем много объектов и они в динамике
3.2 > val, на нем много объектов, что поможет отследить способность модели к обобщению.

Возьмем один на валидацию 

*Подобный подход позволяем себе как раз за счет маленького датасета, который мы в ручную можем отсмотреть и в ручную переименовать. Если бы видео было хотя бы больше 20, то подход бы изменился, на частичный отсмотр, разделение выборки по процентам и автоматическое переименование.*

# Какие классы видит сетка 

1. person
2. bowl
3. cup
4. bottle
5. wine glass

Так же он видит еще овощи и еду, определяет он их по цвету, из-за чего тарелка может стать апельсином или бананом, это мы не трогаем, будем размечать вручную

Так же не будем трогать столовые приборы, чтобы не тратить на это время, при большом желании можно так же найти сетку для 

Из всех этих классов редко появляется bottle и wine glass, таким образом автоматом мы будем размечать только 3 класса 

1. person
2. bowl
3. cup 


In [ ]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLO11 model
model = YOLO(r"runs\detect\train23\weights\best.pt")

# Open the video file
video_path = r"тест\6011752_Person_People_3840x2160.mov"
cap = cv2.VideoCapture(video_path)


screen_res = (1920, 1080) 


max_width, max_height = screen_res

# Store the track history
track_history = defaultdict(lambda: [])

def scale_frame(frame, max_width, max_height):
    height, width = frame.shape[:2]
    scale = min(max_width / width, max_height / height)
    new_width = int(width * scale)
    new_height = int(height * scale)
    return cv2.resize(frame, (new_width, new_height))

# Loop through the video frames
while cap.isOpened():
    success, frame = cap.read()
    if success:
        # Run YOLO11 tracking on the frame, persisting tracks between frames
        result = model.track(frame,
                              persist=True,
                              conf=0.4,
                              iou=0.7,
                             )[0]

        # Get the boxes and track IDs
        if result.boxes and result.boxes.is_track:
            boxes = result.boxes.xywh.cpu()
            track_ids = result.boxes.id.int().cpu().tolist()

            # Visualize the result on the frame
            frame = result.plot()

            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 30 tracks for 30 frames
                    track.pop(0)

                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        frame_resized = scale_frame(frame, max_width, max_height)

        # Display the annotated frame
        cv2.imshow("YOLO11 Tracking", frame_resized)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 5 foods, 4 dishess, 25.9ms
Speed: 2.4ms preprocess, 25.9ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 foods, 3 dishess, 16.9ms
Speed: 2.1ms preprocess, 16.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 foods, 6 dishess, 16.8ms
Speed: 2.4ms preprocess, 16.8ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 foods, 5 dishess, 14.6ms
Speed: 3.3ms preprocess, 14.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 foods, 5 dishess, 15.1ms
Speed: 2.1ms preprocess, 15.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 foods, 6 dishess, 15.9ms
Speed: 2.3ms preprocess, 15.9ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 foods, 6 dishess, 16.4ms
Speed: 2.8ms preprocess, 16.4ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 foods, 6 dishess, 16.6ms
Speed: 2.

# Составим датасет из видео

In [8]:
# Порежем видео на кадры через ffmpeg
# фпс берем маленький, т.к. у нас медленно двигающиеся объекты
# выбираем jpg для экономии места
! ffmpeg -i C:\Users\Администратор\Desktop\zebra_test\dataset\train.mov -vf fps=1.5 C:\Users\Администратор\Desktop\zebra_test\dataset\images_train\0_img%03d.jpg

ffmpeg version 7.1.1-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --enable-mediafound

In [9]:
! ffmpeg -i C:\Users\Администратор\Desktop\zebra_test\dataset\train1.mov -vf fps=1.5 C:\Users\Администратор\Desktop\zebra_test\dataset\images_train\1_img%03d.jpg

ffmpeg version 7.1.1-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --enable-mediafound

In [10]:
! ffmpeg -i C:\Users\Администратор\Desktop\zebra_test\dataset\train2.mov -vf fps=1.5 C:\Users\Администратор\Desktop\zebra_test\dataset\images_train\2_img%03d.jpg

ffmpeg version 7.1.1-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --enable-mediafound

In [11]:
! ffmpeg -i C:\Users\Администратор\Desktop\zebra_test\dataset\train3.mov -vf fps=1.5 C:\Users\Администратор\Desktop\zebra_test\dataset\images_train\3_img%03d.jpg

ffmpeg version 7.1.1-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --enable-mediafound

In [12]:
! ffmpeg -i C:\Users\Администратор\Desktop\zebra_test\dataset\val.mov -vf fps=1.5 C:\Users\Администратор\Desktop\zebra_test\dataset\images_val\1_img%03d.jpg

ffmpeg version 7.1.1-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --enable-mediafound

# Итог по нарезке

Получили 285 в train и 87 на val, соотношение почти 70 на 30, что идеально нам подходит

# Дальше склонируем гит для авторазметки и используем yolo11s.pt, дальше полученный результат в ручную доразметим в интерфейсе cvat


Чтобы настроить конфиг, смотрим видео и просто переносим те классы, которые у нас есть

Вспомним, какие классы мы сейчас будем автоматически размечать 

1. person
2. bowl
3. cup 

Соответственно эти метки переносим в cvat и оставляем в конфиге, остальные метки добавим после автоматической разметки и доделаем руками. 

В конфиге специально делаем маленький conf и большой iou, чтобы получить как можно больше разметки.

запускаем авто разметку следующими командами 

> python AutoCvat.py --img_folder="C:\Users\Администратор\Desktop\zebra_test\AutoCVAT\images_train" --weights=yolo11s.pt --yaml_pth=config.yaml
> python AutoCvat.py --img_folder="C:\Users\Администратор\Desktop\zebra_test\AutoCVAT\images_val" --weights=yolo11s.pt --yaml_pth=config.yaml

Получили очень хорошу разметку, все, что останется сделать это убрать лишние метки и доразметить недостающие.

Опять же делаем упрощение, что у нас маленький датасет и мы можем себе позволить вручную залезть в него и потратить время на подобные операции.


![alt text](img_for_ipynb/2025-06-23_00-55-06.png)
![alt text](img_for_ipynb/2025-06-23_00-57-05.png)

# Оценка по времени 

Тестирование разных моделей и подключение автоматической разметки заняло 2 часа. 

# Разметка оставшихся меток 

В целом все ключевые объекты размечаются и как для теста можно было бы оставить 3 класса. 

Сейчас у нас есть такие классы 

1. person
2. bowl
3. cup

Додавим к этому еще три класса.  

4. food
5. platter 
6. plate 
7. teapot

Таким образом разметить нам нужно будет только 2 класса, т.к. тарелки уже определяются как bowl, а чайник как cup

Добавим соответствующие метки в cvat.


Соусницы автоматически определяются как cup, оставим как есть, соус будем определять как food, что немного расширит вариативность этого класса и даст модели возможность обобщать содержимое даже мелких объектов.

Так же введем упрощение для еды, будем размечать просто внутренности тарелки и считать, все, что внутри = еда, аналогично для соусник и соусов.

Крайне лень размечать все в ручную)) Попробуем запихнуть все классы на разметку и посмотрим результат. 

Залил в grok имена классов и попросил подставить вместо значений 0.01, таким образом сформировал список conf, такой маленький conf необходим, чтобы получить как можно больше классов. 

Маленький iou по той же причине ставим на 0.1.

Не утапливаем до слишком маленьких значений, иначе удалять придется дольше, чем размечали бы руками. 

Получили такой результат ![alt text](2025-06-23_16-14-27.png)

В целом не плохо, но ни один класс в тарелку не попал, а вот на следующих он начал определять мясо как cake, так же увидел broccoli в тарелке и суп определил как cake. Из странного - определил еду на подносе как bowl, ну главное определил. 


![alt text](2025-06-23_16-17-03.png)



# Определим классы, которые встречаются на изображениях через такую авто разметку

1. bowl 
2. cake
3. cup
4. broccoli
5. carrot
6. spoon
7. fork
8. dinning table
9. person
10. sandwich

Из этого бесполезные для нас 

carrot - определяет слишком маленькую область

fork и spoon - в задаче, где мы еду и тарелки определяем нам не нужны приборы. При большом желании их можно отдельно добавить в уже обученную модель, датасеты столовых приборов лежат в свободном доступе кучами. 

Так же уберем все совсем бредовые классы по типу bus, car итд, которые, по понятным причинам, тут появится не могут. 

Отсечем классы мебели, он ее хорошо видит, к задаче отношения не имеет.

Таким образом итоговые классы будут 

1. bowl 
2. cake
3. cup
4. broccoli
8. dinning table
9. person
10. sandwich

Ищем их идентификаторы и делаем следующие переименования 

1. bowl, cup -> dishes. Датасет маленький, поэтому в итоге решил, что не будем разделять на тарелка/поднос/миска итд, просто оставляем как посуда.
2. cake, broccoli, sandwich -> food, по той же причине, к тому же им зададим самый маленький conf, т.к. cake увидел мясо и суп, а sandwich увидел еду на подносе. 

Таким образом почти свели задачу к бинарной классификации, итоговых классов у нас всего 4 будет 

1. food
2. dishes
3. person
4. diinning table

Что уже начинает походить на прикладную задачу cv в ресторанном бизнесе. 


 

Разметка близка к идеальной, в ручную остается только подносы разметить и удалить лишние лейблы.

![alt text](2025-06-23_16-41-02.png)

# Затраты по времени

На разметку трейна ушло примерно час, это с учетом трек полигонов и автоматической разметки большей части изображений. 

*Дополнение по метка*

Можно было добавить еще лейбл с салфетками, что в приложении к ресторанному бизнесу можно было отслеживать качество осблуживания либо трекинг систему для официантов сделать, но у нас слишком мало кадров с салфетками, это можно использовать как идею для будущего развития.

На разметку валидации ушло еще полчаса.

# Посмотрим получившийся датасет

Визуализировать будем через ultralitics hub

In [4]:
# проверим пригодность датасета
from ultralytics.hub import check_dataset

check_dataset("yolo_dataset.zip", task="detect")

Starting HUB dataset checks for C:\Users\\Desktop\zebra_test\yolo_dataset.zip....
WARNING Skipping C:\Users\\Desktop\zebra_test\yolo_dataset.zip unzip as destination directory C:\Users\\Desktop\zebra_test\yolo_dataset is not empty.
Fast image access  (ping: 0.00.0 ms, read: 2785.6442.5 MB/s, size: 274.3 KB)


Scanning C:\Users\Администратор\Desktop\zebra_test\yolo_dataset\labels\train... 285 images, 0 backgrounds, 0 corrupt: 100%|██████████| 285/285 [00:00<00:00, 2162.43it/s]

New cache created: C:\Users\\Desktop\zebra_test\yolo_dataset\labels\train.cache



Statistics: 100%|██████████| 285/285 [00:00<00:00, 281994.96it/s]

Fast image access  (ping: 0.00.0 ms, read: 2872.4505.3 MB/s, size: 236.1 KB)



Scanning C:\Users\Администратор\Desktop\zebra_test\yolo_dataset\labels\val... 87 images, 0 backgrounds, 0 corrupt: 100%|██████████| 87/87 [00:00<00:00, 2468.44it/s]

New cache created: C:\Users\\Desktop\zebra_test\yolo_dataset\labels\val.cache



Statistics: 100%|██████████| 87/87 [00:00<00:00, 244083.24it/s]

Checks completed correctly . Upload this dataset to https://hub.ultralytics.com/datasets/.


# Все подходит, загрузим через ui хаба

По изображениям достаточно сбалансированно получилось

![alt text](2025-06-23_19-04-21.png)

А вот по инстансам очень сильный дисбаланс, т.к. я не фиксил сильно огрехи авторазметки и получалось, что на одном кадре могло быть по 5-10 полигонов для еды и аналогично для посуды. 

![alt text](2025-06-23_19-06-48.png)

# Вывод

Хоть на инстансах у нас и огромный дисбаланс, но соотношение еды к посуде 6390 к 4377, что в целом не сильно критично, а если person и dining table начнут тянуть метрики вниз, то просто отрежем их и для этих целей будем использовать предобученную модель. 

Затестим базовую yolo11s на этих данных и посмотрим динамику роста метрик, после чего используем встроенный тюнер и потестим аугментацию. Если аугментация не даст прироста, попробуем дозаполнить синтетикой. 

# Тест базовой модели без доп настроек на полученной датасете.

In [7]:
from ultralytics import YOLO
from clearml import Task, Logger

In [8]:
# версионируем все через clearml
task = Task.init(
    project_name='zebra_test', 
    task_name='BaseYolo11', 
    tags=['yolo11, base args, no augment ds'])

ClearML Task: created new task id=f7af6cd6f70241fc9d866a78b4fac14f
ClearML results page: https://app.clear.ml/projects/45d33c510f314a938c8dc1ea8644e9d4/experiments/f7af6cd6f70241fc9d866a78b4fac14f/output/log


2025-06-23 19:14:25,485 - clearml.storage - INFO - Uploading: 91.41MB from C:/Users/836D~1/AppData/Local/Temp/yolo_dataset.44ulbile.zip


████████████████▉                55% | 49.94/91.41 MB [01:28<01:20,  1.93s/MB]: 

ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


███████████████████████████████ 100% | 91.41/91.41 MB [02:50<00:00,  1.87s/MB]: 


In [9]:
# загрузим датасет
task.upload_artifact(name='data.raw', artifact_object="yolo_dataset")

True

In [10]:
model = YOLO("yolo11s.pt")

In [ ]:
model.train(data="yolo_dataset\yolo_dataset.yaml",
            epochs=10,
            batch=-1,
            imgsz=640,
            device=0,
            cache=False,
            workers=4,
            )

<>:1: SyntaxWarning: invalid escape sequence '\y'
<>:1: SyntaxWarning: invalid escape sequence '\y'
C:\Users\Администратор\AppData\Local\Temp\ipykernel_14760\3700638081.py:1: SyntaxWarning: invalid escape sequence '\y'
  model.train(data="yolo_dataset\yolo_dataset.yaml",


Ultralytics 8.3.158  Python-3.13.1 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_dataset\yolo_dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0

train: Scanning C:\Users\Администратор\Desktop\zebra_test\yolo_dataset\labels\train.cache... 285 images, 0 backgrounds, 0 corrupt: 100%|██████████| 285/285 [00:00<?, ?it/s]

AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 3090) 24.00G total, 0.13G reserved, 0.10G allocated, 23.77G free


      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     9429340       21.55         0.835         65.73         496.6        (1, 3, 640, 640)                    list
     9429340       43.11         1.181         40.94         172.9        (2, 3, 640, 640)                    list
     9429340       86.22         1.755         34.74         174.4        (4, 3, 640, 640)                    list
     9429340       172.4         2.858         52.54         171.2        (8, 3, 640, 640)                    list
     9429340       344.9         4.962         92.74         173.7       (16, 3, 640, 640)                    list
     9429340       689.8         9.387         163.6         196.6       (32, 3, 640, 640)                    list
     9429340        1380        17.941         321.6           244       (64, 3, 640, 640)                    list
AutoBatch: Using batch-size 49 for CUDA:0 14.15G/24.00G (59%) 
train: Fast image

...

2025-06-23 19:19:05,723 - clearml.storage - INFO - Uploading: 18.26MB from runs\detect\train\weights\best.pt


███████████████████████████████ 100% | 18.26/18.26 MB [00:26<00:00,  1.44s/MB]: 

2025-06-23 19:19:32,103 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/zebra_test/BaseYolo11.f7af6cd6f70241fc9d866a78b4fac14f/models/best.pt


███████████████████████████████ 100% | 18.35/18.35 MB [00:26<00:00,  1.46s/MB]: 

2025-06-23 19:55:52,466 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/zebra_test/BaseYolo11.f7af6cd6f70241fc9d866a78b4fac14f/models/best.pt


In [22]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLO11 model
model = YOLO(r"runs\detect\train\weights\best.pt")

# Open the video file
# video_path = r"dataset\test.mov"
video_path = r"тест\6011752_Person_People_3840x2160.mov"
cap = cv2.VideoCapture(video_path)

# Получаем размер экрана (монитора)
screen_res = (1920, 1080)  # Можно заменить на реальный размер экрана, например с помощью pyautogui или ctypes
# Альтернативный способ получить разрешение экрана (для Windows):
# import ctypes
# user32 = ctypes.windll.user32
# screen_res = (user32.GetSystemMetrics(0), user32.GetSystemMetrics(1))

max_width, max_height = screen_res

# Store the track history
track_history = defaultdict(lambda: [])

def scale_frame(frame, max_width, max_height):
    height, width = frame.shape[:2]
    scale = min(max_width / width, max_height / height)
    new_width = int(width * scale)
    new_height = int(height * scale)
    return cv2.resize(frame, (new_width, new_height))

# Loop through the video frames
while cap.isOpened():
    success, frame = cap.read()
    if success:
        # Run YOLO11 tracking on the frame, persisting tracks between frames
        result = model.track(frame,
                              persist=True,
                              conf=0.5,
                             )[0]

        # Get the boxes and track IDs
        if result.boxes and result.boxes.is_track:
            boxes = result.boxes.xywh.cpu()
            track_ids = result.boxes.id.int().cpu().tolist()

            # Visualize the result on the frame
            frame = result.plot()

            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 30 tracks for 30 frames
                    track.pop(0)

                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # Масштабируем кадр под экран
        frame_resized = scale_frame(frame, max_width, max_height)

        # Display the annotated frame
        cv2.imshow("YOLO11 Tracking", frame_resized)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 8 foods, 3 persons, 15 dishess, 91.2ms
Speed: 2.0ms preprocess, 91.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 foods, 3 persons, 14 dishess, 10.0ms
Speed: 1.6ms preprocess, 10.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 foods, 3 persons, 12 dishess, 10.3ms
Speed: 1.7ms preprocess, 10.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 foods, 3 persons, 13 dishess, 10.0ms
Speed: 1.7ms preprocess, 10.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 foods, 3 persons, 12 dishess, 10.5ms
Speed: 1.7ms preprocess, 10.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 foods, 2 persons, 11 dishess, 10.6ms
Speed: 1.7ms preprocess, 10.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 foods, 2 persons, 16 dishess, 21.1ms
Speed: 2.0ms preprocess, 21.1ms inference, 3.8ms postproces

# Итог 

В целом модель полностью готова, на тестовом видео не видит только одно блюдо в центре, из-за того, что я его проморгал каким то образом на разметке, так же начинают мелькать лишние детали из-за качества разметки, я не чистил мелкие полигоны авторазметки для экономии времени. 

Что касается качества результата, все упирается в разметку

# Оценка по времени 

Дойти до этого момента заняло 4 часа

2 часа на препроцессинг (получение датасета, тесты моделей для авторазметки, нарезка видео, настройка авторазметки)
1:30 часа на разметку
30 минут на подготовку датасета и версионирование. 

# Перед более подробным подбором гиперпараметров увеличим количество эпох и размер изображений

In [23]:
model.train(data="yolo_dataset\yolo_dataset.yaml",
            epochs=40,
            batch=-1,
            imgsz=1280,
            device=0,
            cache=False,
            workers=4,
            )

<>:1: SyntaxWarning:

invalid escape sequence '\y'

<>:1: SyntaxWarning:

invalid escape sequence '\y'

C:\Users\Администратор\AppData\Local\Temp\ipykernel_14760\1589773017.py:1: SyntaxWarning:

invalid escape sequence '\y'



Ultralytics 8.3.158  Python-3.13.1 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_dataset\yolo_dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs\detect\train\weights\best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0

train: Scanning C:\Users\Администратор\Desktop\zebra_test\yolo_dataset\labels\train.cache... 285 images, 0 backgrounds, 0 corrupt: 100%|██████████| 285/285 [00:00<?, ?it/s]

AutoBatch: Computing optimal batch size for imgsz=1280 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 3090) 24.00G total, 1.21G reserved, 0.57G allocated, 22.22G free


      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     9429340       86.22         1.961         117.5         137.1      (1, 3, 1280, 1280)                    list
     9429340       172.4         3.217          46.6         135.8      (2, 3, 1280, 1280)                    list
     9429340       344.9         5.482         85.61         135.8      (4, 3, 1280, 1280)                    list
     9429340       689.8        10.177         166.9         149.3      (8, 3, 1280, 1280)                    list
     9429340        1380        19.258         752.1         289.4     (16, 3, 1280, 1280)                    list
     9429340        2759        37.430      1.29e+04     6.661e+04     (32, 3, 1280, 1280)                    list
CUDA out of memory. Tried to allocate 50.00 MiB. GPU 0 has a total capacity of 24.00 GiB of which 0 bytes is free. Of the allocated memory 35.58 GiB is allocated by PyTorch, and 227.59 MiB is res

...


40 epochs completed in 0.115 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 19.2MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 19.2MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.158  Python-3.13.1 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
YOLO11s summary (fused): 100 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  2.75it/s]


                   all         87       2611      0.795       0.76      0.741      0.514
                  food         87       1121      0.563      0.581      0.518      0.233
                person         75         87      0.633      0.667      0.616      0.418
          dining table         87         87      0.996          1      0.995      0.796
                dishes         87       1316      0.987      0.792      0.835       0.61
Speed: 0.3ms preprocess, 6.1ms inference, 0.0ms loss, 9.3ms postprocess per image
Results saved to runs\detect\train2


2025-06-23 19:55:15,686 - clearml.storage - INFO - Starting upload: runs\detect\train2\weights\best.pt => https://files.clear.ml/zebra_test/BaseYolo11.f7af6cd6f70241fc9d866a78b4fac14f/models/best.pt
2025-06-23 19:55:25,740 - clearml.storage - INFO - Uploading: 18.35MB from runs\detect\train2\weights\best.pt


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001D0570BC440>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
       

# Увеличение по размерам не дало прирост метрик, будем тюнить на 640